In [1]:
import os
import pytz
import pandas as pd
import geopandas as gpd
import contextily as ctx
from datetime import datetime
import matplotlib.pyplot as plt
from shapely.geometry import Point
from matplotlib import rcParams

# Headers for requests
headers = {
    'accept': 'application/json, text/plain, */*',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
}

# Set timezone to UTC and then convert to America/Los_Angeles
utc_now = datetime.utcnow().replace(tzinfo=pytz.utc)
la_now = utc_now.astimezone(pytz.timezone('America/Los_Angeles'))

# Get current week number and date in Los Angeles time
week_number = la_now.isocalendar().week
today_la = la_now.strftime('%Y-%m-%d')

# Use timestamp for cacheBust
cache_bust = int(la_now.timestamp())

# Format the URL dynamically
url = f'https://data.lacity.org/api/views/e7h6-4a3e/rows.csv?fourfour=s49e-q6j2&cacheBust={cache_bust}&date={today_la}&accessType=DOWNLOAD'

In [2]:
locations = pd.read_csv(url, storage_options=headers)  # Ensure headers are being used if required

In [3]:
# Convert the string to UTC datetime specifying the format
locations['EventTime_UTC'] = pd.to_datetime(locations['EventTime_UTC'], format='%m/%d/%Y %I:%M:%S %p', utc=True)

# Convert from UTC to Los Angeles time
locations['EventTime_LA'] = locations['EventTime_UTC'].dt.tz_convert('America/Los_Angeles')

In [4]:
locations.columns = locations.columns.str.lower()
locations_df = locations.sort_values('eventtime_la', ascending=False).copy()

In [5]:
locations_df.head()

,spaceid,eventtime_utc,occupancystate,eventtime_la
3676,SH398,2024-09-01 22:36:17+00:00,VACANT,2024-09-01 15:36:17-07:00
3738,SV404,2024-09-01 22:36:08+00:00,VACANT,2024-09-01 15:36:08-07:00
629,SV332,2024-09-01 22:35:53+00:00,VACANT,2024-09-01 15:35:53-07:00
4208,SH386,2024-09-01 22:35:46+00:00,VACANT,2024-09-01 15:35:46-07:00
1353,WV280,2024-09-01 22:35:46+00:00,VACANT,2024-09-01 15:35:46-07:00


---

In [6]:
inventory_gdf = gpd.read_file('data/parking_meters_inventory_latest.geojson')

In [7]:
inventory_gdf.head()

,spaceid,blockface,metertype,ratetype,raterange,meteredtimelimit,name,city,region,latitude,longitude,geometry
0,WW1218,4200 3RD ST,Single-Space,FLAT,$1.00,1HR,Koreatown,los-angeles,central-la,34.068886,-118.306322,POINT (-118.30632 34.06889)
1,VW1958,1701 HOOVER ST,Single-Space,FLAT,$1.00,1HR,Pico-Union,los-angeles,central-la,34.042304,-118.284271,POINT (-118.28427 34.04230)
2,CB2210,801 S SAN JULIAN ST,Single-Space,TOD,$0.50 - $6.00,2HR,Downtown,los-angeles,central-la,34.038399,-118.250914,POINT (-118.25091 34.03840)
3,AE412,500 VIGNES ST,Single-Space,JUMP,$1.00/H - $4.00/10H,10HR,Downtown,los-angeles,central-la,34.052172,-118.233119,POINT (-118.23312 34.05217)
4,W190,5300 W CENTURY BLVD,Single-Space,FLAT,$1.00,2HR,Westchester,los-angeles,south-bay,33.945157,-118.371691,POINT (-118.37169 33.94516)


---

In [8]:
gdf = gpd.GeoDataFrame(pd.merge(inventory_gdf, locations_df, on='spaceid'))

In [9]:
# Define the mapping from neighborhood to area
area_mapping = {
    'Marina del Rey': 'Beach',
    'Venice': 'Beach',
    'Downtown': 'Downtown',
    'Chinatown': 'Downtown',
    'Hollywood': 'Hollywood',
    'Elysian Park': 'Dodgers',
    'San Pedro': 'Harbor',
    'Westwood': 'UCLA'
}

# Apply the mapping to create a new 'Area' column
gdf['region'] = gdf['name'].map(area_mapping)

In [10]:
places = list(gdf['name'].unique())
data_list = []

for place in places:
    place_gdf = gdf[gdf['name'] == place]
    total_spaces = len(place_gdf)
    counts = place_gdf['occupancystate'].value_counts()
    percentages = counts / total_spaces * 100  # Calculate percentages
    
    # Create a data row for this region
    data_row = {
        'region': place,
        'spaces': total_spaces,
        'unknown': counts.get('UNKNOWN', 0),
        'occupied': counts.get('OCCUPIED', 0),
        'vacant': counts.get('VACANT', 0),
        'ukn_pct': percentages.get('UNKNOWN', 0),
        'occ_pct': percentages.get('OCCUPIED', 0),
        'vac_pct': percentages.get('VACANT', 0)
    }
    data_list.append(data_row)

# Convert list of data rows to DataFrame
summary_df = pd.DataFrame(data_list).round(2)

In [11]:
summary_df['fetched'] = pd.to_datetime(la_now).strftime('%Y-%m-%d %H:%M:%S')

In [12]:
summary_df

,region,spaces,unknown,occupied,vacant,ukn_pct,occ_pct,vac_pct,fetched
0,Downtown,3897,0,2439,1458,0.00,62.59,37.41,2024-09-01 15:36:58
1,Hollywood,778,28,381,369,3.60,48.97,47.43,2024-09-01 15:36:58
2,Marina del Rey,29,26,1,2,89.66,3.45,6.90,2024-09-01 15:36:58
3,Chinatown,336,0,206,130,0.00,61.31,38.69,2024-09-01 15:36:58
4,Westwood,400,0,308,92,0.00,77.00,23.00,2024-09-01 15:36:58
5,Venice,180,73,72,35,40.56,40.00,19.44,2024-09-01 15:36:58
6,San Pedro,305,0,193,112,0.00,63.28,36.72,2024-09-01 15:36:58
7,Elysian Park,15,0,12,3,0.00,80.00,20.00,2024-09-01 15:36:58


In [ ]:
# Setting the font globally
rcParams['font.family'] = 'Roboto'

# Ensure the visuals directory exists
visuals_dir = 'visuals'
os.makedirs(visuals_dir, exist_ok=True)

for place in places:
    place_gdf = gdf.query(f'name == "{place}"').copy()
    place_gdf = place_gdf.to_crs(epsg=3857)  # Convert to Web Mercator

    # Create a dictionary to map occupancystate to colors
    color_map = {
        'UNKNOWN': '#666',
        'OCCUPIED': '#FF474C',
        'VACANT': '#90EE90'
    }
    
    # Map the colors to your dataframe
    place_gdf['color'] = place_gdf['occupancystate'].map(color_map)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    # Plot the data using longitude and latitude as X and Y
    scatter = ax.scatter(x=place_gdf.geometry.x, y=place_gdf.geometry.y, alpha=1, s=20, c=place_gdf['color'], label='Occupancy State')
    
    # Add the basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    
    # Adjust the axis to fit the plot (this fixes any misalignment issues)
    ax.set_xlim(place_gdf.geometry.bounds.minx.min(), place_gdf.geometry.bounds.maxx.max())
    ax.set_ylim(place_gdf.geometry.bounds.miny.min(), place_gdf.geometry.bounds.maxy.max())

    # Remove the axis
    ax.set_axis_off()

    # Setting title with specific font
    ax.set_title(f'{place} Parking Meter Locations', fontsize=14)
    
    # Create a custom legend
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label='Unknown', markersize=10, markerfacecolor='#666'),
                       plt.Line2D([0], [0], marker='o', color='w', label='Occupied', markersize=10, markerfacecolor='#FF474C'),
                       plt.Line2D([0], [0], marker='o', color='w', label='Vacant', markersize=10, markerfacecolor='#90EE90')]
    ax.legend(handles=legend_elements, title='Occupancy State')

    # Save the figure
    fig_path = os.path.join(visuals_dir, f'{place.replace(" ", "_").lower()}_parking_meter_locations.png')
    plt.savefig(fig_path, format='png', dpi=300)
    plt.close(fig)  # Close the plot to free up memory